# 13.3 Observability（可观测）实验

本笔记本演示如何为 LLM 应用添加日志和监控。

**预估成本**: ~$0.01

In [ ]:
!pip install openai -q

In [ ]:
import os
import json
import time
from datetime import datetime
from openai import OpenAI

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

## Demo: LLM 日志系统

In [ ]:
class LLMLogger:
    def __init__(self, log_file='llm_calls.jsonl'):
        self.log_file = log_file
    
    def log_call(self, input_text, output_text, model, prompt_tokens, completion_tokens, latency_ms, **metadata):
        log_entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'model': model,
            'input': input_text,
            'output': output_text,
            'prompt_tokens': prompt_tokens,
            'completion_tokens': completion_tokens,
            'total_tokens': prompt_tokens + completion_tokens,
            'latency_ms': round(latency_ms, 2),
            **metadata
        }
        
        # 计算成本
        if model == 'gpt-4o-mini':
            cost = (prompt_tokens * 0.15 + completion_tokens * 0.6) / 1_000_000
        else:
            cost = 0
        log_entry['cost_usd'] = round(cost, 6)
        
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(log_entry, ensure_ascii=False) + '\n')

logger = LLMLogger()

def tracked_llm_call(question):
    start = time.time()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': question}]
    )
    latency = (time.time() - start) * 1000
    
    output = response.choices[0].message.content
    logger.log_call(
        input_text=question,
        output_text=output,
        model='gpt-4o-mini',
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        latency_ms=latency
    )
    return output

# Test
for q in ['什么是Python？', '什么是Docker？']:
    print(f'Q: {q}')
    print(f'A: {tracked_llm_call(q)[:100]}...\n')